In [19]:
# Cell 0
# If at all possible please test locally or on the private tbears server. The Testnet
# is becoming cluttered with many deployments of Balanced contracts.
# Note that running on the private tbears server will require the number of top P-Reps 
# be set to 4 in the staking contract or it will fail to deploy.

network = "custom"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"gangnam": {"iconservice": "https://gicon.net.solidwallet.io", "nid": 3},
"hannam": {"iconservice": "https://hannam.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.144.108.38:9000",        "nid": 3}}

env = connections[network]

In [20]:
# Cell 1
import sys
import time
sys.path.append("..")
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import pickle as pkl
from datetime import datetime
from time import sleep
import json
import os


ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
TEST_ORACLE = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"
MAIN_ORACLE = "cxe647e0af68a4661566f5e9861ad4ac854de808a2"
BALANCED_TEST = "hx3f01840a599da07b0f620eeae7aa9c574169a4be"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [21]:
# Cell 2

icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [22]:
# Cell 3

wallet = KeyWallet.load("../keystores/keystore_test1.json", "test1_Account")
# Balanced test wallet
with open("../keystores/balanced_test.pwd", "r") as f:
    key_data = f.read()
btest_wallet = KeyWallet.load("../keystores/balanced_test.json", key_data)
with open("../keystores/staking_test.pwd", "r") as f:
    key_data = f.read()
staking_wallet = KeyWallet.load("../keystores/staking_test.json", key_data)


In [5]:
print(wallet.get_address())
print(icon_service.get_balance(wallet.get_address()) / 10**18)

hxe7af5fcfd8dfc67530a01a0e403882687528dfcb
778732131.2056184


In [6]:
print(btest_wallet.get_address())
print(icon_service.get_balance(btest_wallet.get_address()) / 10**18)

hx3f01840a599da07b0f620eeae7aa9c574169a4be
1674752.2121612


In [7]:
user1 = KeyWallet.load("../keystores/user1.json","HelloWorld@1234")
# btest_wallet = KeyWallet.load("./balanced_test.json","HelloWorld@1234")

print(icon_service.get_balance(user1.get_address())/10**18)
print(user1.get_address())

# test2 = hx7a1824129a8fe803e45a3aae1c0e060399546187
private = "0a354424b20a7e3c55c43808d607bddfac85d033e63d7d093cb9f0a26c4ee022"
user2 = KeyWallet.load(bytes.fromhex(private))
print(icon_service.get_balance(user2.get_address())/10**18)
print(user2.get_address())

173999.7256714
hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db
206479.4400324
hx7a1824129a8fe803e45a3aae1c0e060399546187


In [8]:
dict_list = []
for i in range(0, 5):
    data = {}
    wallet2 = KeyWallet.create()
    address = wallet2.get_address()
    private_key = wallet2.get_private_key()
    data[address] = private_key
    dict_list.append(data)
with open('wallet2.json', 'a') as f:
    json.dump(dict_list, f, ensure_ascii=False, indent=4)

In [10]:
with open('wallet2.json') as f:
    data = json.load(f)

In [ ]:
for dict in data:
    for items in dict.items():
        print(items[0])
        set_div_per = TransactionBuilder() \
            .from_(wallet.get_address()) \
            .to(items[0]) \
            .nid(NID) \
            .nonce(100) \
            .value(5000 * 10 ** 18) \
            .build()
        estimate_step = icon_service.estimate_step(set_div_per)
        step_limit = estimate_step + 10000000
        signed_transaction = SignedTransaction(set_div_per, wallet, step_limit)

        tx_hash = icon_service.send_transaction(signed_transaction)
        ab = get_tx_result(tx_hash)

In [23]:
# Cell 5a
# The following addresses are those deployed to the private tbears server.

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cxa0f715fb2c4bc8f4c6399c2cc26167a27be0aa61'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxbabed822d59b605dbeb6322735c529b292baac3b'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx1379084f45776301abda3849c6e374f460ee0155'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx71dda2221bf88faddc8f84b72ffd6db296e5609e'},
 'daofund': {'zip': 'core_contracts/daofund.zip',
  'SCORE': 'cxfd09787f23d23b945fa0c7eb55b5aa69425da1c8'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx27aa3bf62145822e60d85fa5d18dabdcff5b9ada'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx01eee12b6614e5328e0a84261652cb7f055e0176'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cxd7b3e71dcff3d75392216e208f28ef68e8a54ec0'},
 'oracle': {'zip': 'core_contracts/oracle.zip',
  'SCORE': 'cxed97bdb35a7ca1b3993e400e4dba9e11610338f7'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx799f724e02560a762b5f2bd3b6d2d8d59d7aecc1'},
 'bnUSD': {'zip': 'token_contracts/bnUSD.zip',
  'SCORE': 'cx266bdc0c35828c8130cdf1cbaa3ad109f7694722'},
 'bnXLM': {'zip': 'token_contracts/bnXLM.zip',
  'SCORE': 'cx266bdc0c35828c8130cdf1cbaa3ad109f7694722'},
 'bnDOGE': {'zip': 'token_contracts/bnDOGE.zip',
  'SCORE': 'cx266bdc0c35828c8130cdf1cbaa3ad109f7694722'},
 'baln': {'zip': 'token_contracts/baln.zip',
  'SCORE': 'cx4d0768508a7ff550de4405f27aebfb8831565c19'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx663f9d59163846d9f6c6f7b586858c59aa8878a9'}}

In [24]:
# Cell 6
# Define deploy and send_tx functions

def send_icx(to, amount, wallet):
    transaction = TransactionBuilder()\
        .from_(wallet.get_address())\
        .to(to)\
        .value(amount)\
        .step_limit(1000000) \
        .nid(NID) \
        .nonce(2) \
        .version(3) \
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    return icon_service.send_transaction(signed_transaction)


def compress():
    """
    Compress all SCORE folders in the core_contracts and token_contracts folders.
    Make sure the oracle address is correct.
    """
    deploy = list(contracts.keys())[:]
    for directory in {"../../core_contracts", "../../token_contracts"}:
        with os.scandir(directory) as it:
            for file in it:
                archive_name = directory + "/" + file.name
                if file.is_dir() and file.name in deploy:
                    make_archive(archive_name, "zip", directory, file.name)
                    contracts[file.name]['zip'] = archive_name + '.zip'
    if network == "yeouido":
        contracts['oracle']['SCORE'] = TEST_ORACLE
    elif network == "mainnet":
        contracts['oracle']['SCORE'] = MAIN_ORACLE
                    

def deploy_SCORE(contract, params, wallet, update) -> str:
    """
    contract is of form {'zip': 'core_contracts/governance.zip', 'SCORE': 'cx1d81f93b3b8d8d2a6455681c46128868782ddd09'}
    params is a dicts
    wallet is a wallet file
    update is boolian
    """
    print(f'{contract["zip"]}')
    if update:
        dest = contract['SCORE']
    else:
        dest = GOVERNANCE_ADDRESS
    zip_file = contract['zip']
    step_limit = 3000000000
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(dest)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(zip_file))\
        .params(params)\
        .build()

    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    print('')
    return res

def send_tx(dest, value, method, params, wallet, _print = True):
    """
    dest is the name of the destination contract.
    """
    if _print:
        print('------------------------------------------------------------------------------------------------------------------')
        print(f'Calling {method}, with parameters {params} on the {dest} contract.')
        print('------------------------------------------------------------------------------------------------------------------')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .value(value)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(method)\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    if _print:
        print(f'************************************************** Status: {res["status"]} **************************************************')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        if _print:
            print(f'Failure: {res["failure"]}')
    return res

def deploy_all(wallet, staking_wallet):
    """
    Compress and Deploy all SCOREs.
    """
    compress()

    deploy = list(contracts.keys())[:]
    deploy.remove('oracle')
    deploy.remove('staking')
    deploy.remove('sicx')
    deploy.remove('governance')
    if network == "mainnet":
        if 'bnXLM' in deploy:
            deploy.remove('bnXLM')
        if 'bnDOGE' in deploy:
            deploy.remove('bnDOGE')

    results = {}
    res = deploy_SCORE(contracts['governance'], {}, wallet, 0)
    results[f'{contracts["governance"]}|deploy|{{}}'] = res
    governance = res.get('scoreAddress', '')
    contracts['governance']['SCORE'] = governance
    params = {'_governance': governance}
    for score in deploy:
        res = deploy_SCORE(contracts[score], params, wallet, 0)
        results[f'{contracts[score]}|deploy|{params}'] = res
        contracts[score]['SCORE'] = res.get('scoreAddress', '')

    res = deploy_SCORE(contracts['staking'], {}, staking_wallet, 0)
    results[f'{contracts["staking"]}|deploy|{{}}'] = res
    contracts['staking']['SCORE'] = res.get('scoreAddress', '')

    params = {'_admin': contracts['staking']['SCORE']}
    res = deploy_SCORE(contracts['sicx'], params, staking_wallet, 0)
    results[f'{contracts["sicx"]}|deploy|{params}'] = res
    contracts['sicx']['SCORE'] = res.get('scoreAddress', '')
    
    return results

    
def config_balanced(wallet, staking_wallet):
    """
    Configure all SCOREs before launch.
    """
    config = list(contracts.keys())[:]
    config.remove('governance')
    config.remove('bnDOGE')
    config.remove('bnXLM')
    addresses = {contract: contracts[contract]['SCORE'] for contract in config}
    txns = [{'contract': 'governance', 'value': 0, 'method': 'setAddresses', 'params': {'_addresses': addresses}, 'wallet': wallet},
            {'contract': 'staking', 'value': 0, 'method': 'setSicxAddress', 'params': {'_address': contracts['sicx']['SCORE']}, 'wallet': staking_wallet},
            {'contract': 'governance', 'value': 0, 'method': 'configureBalanced', 'params': {}, 'wallet': wallet}]

    results = {}
    for tx in txns:
        res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], tx["wallet"])
        results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

    return results

def launch_balanced(wallet, staking_wallet):
    """
    Launch Balanced, turn on staking management, and set delegation for sICX on the Loans contract.
    """
    if network == "custom":
        preps = {
            "hx9eec61296a7010c867ce24c20e69588e2832bc52",  # ICX Station
            "hx000e0415037ae871184b2c7154e5924ef2bc075e"}  # iBriz-ICONOsphere
    elif network == "yeouido":
        preps = {
            "hx23823847f593ecb65c9e1ea81a789b02766280e8",  # ICX Station
            "hxe0cde6567eb6529fe31b0dc2f2697af84847f321",  # iBriz-ICONOsphere
            "hx83c0fc2bcac7ecb3928539e0256e29fc371b5078",  # Mousebelt
            "hx48b4636e84d8c491c88c18b65dceb7598c4600cc",  # Parrot 9
            "hxb4e90a285a79687ec148c29faabe6f71afa8a066"}  # ICONDAO
    elif network == "mainnet":
        preps = {
            "hxfba37e91ccc13ec1dab115811f73e429cde44d48",  # ICX Station
            "hx231a795d1c719b9edf35c46b9daa4e0b5a1e83aa",  # iBriz-ICONOsphere
            "hxbc9c73670c79e8f6f8060551a792c2cf29a8c491",  # Mousebelt
            "hx28c08b299995a88756af64374e13db2240bc3142"}  # Parrot 9
    else:
        return

    txns = [{'contract': 'governance', 'value': 0, 'method': 'launchBalanced', 'params': {}, 'wallet': wallet},
            {'contract': 'staking', 'value': 0, 'method': 'toggleStakingOn', 'params': {}, 'wallet': staking_wallet},
            {'contract': 'governance', 'value': 0, 'method': 'delegate', 'params': {'_delegations': [{'_address': prep, '_votes_in_per': str(100 * ICX // len(preps))} for prep in preps]}, 'wallet': wallet}]

    results = {}
    for tx in txns:
        res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], tx["wallet"])
        results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

    return results
   

def get_scores_json(contracts):
    """
    Prints out dictionary of SCORE addresses for use in testing UI.
    """
    scores = {}
    for score in contracts:
        scores[score] = contracts[score]['SCORE']
    return json.dumps(scores)

def call_tx(dest: str, method: str, params: dict = {}, _print = True):
    """
    dest is the name of the destination contract.
    """
    if _print:
        print('------------------------------------------------------------------------------------------------------------------')
        print(f'Reading {method}, with parameters {params} on the {dest} contract.')
        print('------------------------------------------------------------------------------------------------------------------')
    call = CallBuilder()\
        .from_(btest_wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .method(method)\
        .params(params)\
        .build()
    result = icon_service.call(call)
    if _print:
        print(result)
    return result

def fast_send_tx(dest, value, method, params, wallet, _print = False):
    """
    dest is the name of the destination contract.
    """
    if _print:
        print('------------------------------------------------------------------------------------------------------------------')
        print(f'Calling {method}, with parameters {params} on the {dest} contract.')
        print('------------------------------------------------------------------------------------------------------------------')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .value(value)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(method)\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    
    res = get_tx_result(tx_hash)
    if _print:
        print(f'************************************************** Status: {res["status"]} **************************************************')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        if _print:
            print(f'Failure: {res["failure"]}')
    return res



In [25]:
# Cell 7
# Deploy and configure Balanced. Print results if anything goes wrong.

if network == 'custom':
    confirm = 'Yes'
else:
    confirm = input(f'Deploying Balanced to {network}. Proceed (Yes/No)? ')
if confirm == 'Yes':
    results = {}
    deploy_all(btest_wallet, staking_wallet)
    print('------------------------------------------------------------------------------------------------------------------')
    print(contracts)
    print('----------Contracts for Testing UI--------------------------------------------------------------------------------')
    print(get_scores_json(contracts))

../../core_contracts/governance.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

../../core_contracts/loans.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

../../core_contracts/dividends.zip


...

Retrying in 1 seconds...
Status: 1

../../token_contracts/bwt.zip
Status: 1

../../core_contracts/staking.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

../../token_contracts/sicx.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

------------------------------------------------------------------------------------------------------------------
{'loans': {'zip': '../../core_contracts/loans.zip', 'SCORE': 'cxedda469d71f9d3cb76b5585e129a374f988684bc'}, 'staking': {'zip': '../../core_contracts/staking.zip', 'SCORE': 'cx272ce34980099a03c2465c8de6f445ff86416793'}, 'dividends': {'zip': '../../core_contracts/dividends.zip', 'SCORE': 'cx37854d456b9095ccd82b39f5e3f0e60e6e7577f1'}, 'reserve': {'zip': '../../core_contracts/reserve.zip', 'SCORE': 'cxffa4b54daff78fe6cbb367dbea6fbd077e86c77b'}, 'daofund': {'zip': '../../core_contracts/daofund.zip', 'SCORE': 'cx7f5b9527a5186ae60262bad895d677c74d4782f3'}, 'rewards': {'zip': '../../core_contracts/rewards.zip', 'SCORE': 'cxc1b3632a24bdb243620bbffbf1b8dc8175346a27'}, 'dex': {'zip': '../../core_contracts/dex.zip', 'SCORE': 'cxa2b16e0314387d558ee8c4af02b48f611aff8be9'}, 'governance': {'zip': '../../core_contracts/governance.zip', 'SCORE': 'cx0f5d67

In [26]:
# Cell 7a
# Configure Balanced

config_results = config_balanced(btest_wallet, staking_wallet)
print(config_results)

------------------------------------------------------------------------------------------------------------------
Calling setAddresses, with parameters {'_addresses': {'loans': 'cxedda469d71f9d3cb76b5585e129a374f988684bc', 'staking': 'cx272ce34980099a03c2465c8de6f445ff86416793', 'dividends': 'cx37854d456b9095ccd82b39f5e3f0e60e6e7577f1', 'reserve': 'cxffa4b54daff78fe6cbb367dbea6fbd077e86c77b', 'daofund': 'cx7f5b9527a5186ae60262bad895d677c74d4782f3', 'rewards': 'cxc1b3632a24bdb243620bbffbf1b8dc8175346a27', 'dex': 'cxa2b16e0314387d558ee8c4af02b48f611aff8be9', 'oracle': 'cxed97bdb35a7ca1b3993e400e4dba9e11610338f7', 'sicx': 'cx2f4b75d1eba58dc41dd45b3b1d09d33319f751c5', 'bnUSD': 'cxdc49b3be43c4733f83a2fe80ccbe6fee11c8ba12', 'baln': 'cx23abbd450911b5f7feedda2de23621ac765622fb', 'bwt': 'cx03198e44c6dff023caa943a543fad9a9ac63aa3a'}} on the governance contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
------------------------------------------------------------------------------------------------------------------
Calling setSicxAddress, with parameters {'_address': 'cx2f4b75d1eba58dc41dd45b3b1d09d33319f751c5'} on the staking contract.
------------------------------------------------------------------------------------------------------------------
************************************************** Status: 1 **************************************************
------------------------------------------------------------------------------------------------------------------
Calling configureBalanced, with parameters {} on the governance contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxedda469d71f9d3cb76b5585e129a374f988684bc', 'indexed': ['AssetAdded(Address,str,bool)', 'cx2f4b75d1eba58dc41dd45b3b1d09d33319f751c5', 'sICX', '0x1'], 'data': []} 

{'scoreAddress': 'cxedda469d71f9d3cb76b5585e129a374f988684bc', 'indexed': ['AssetAdded(Address,str,bool)', 'cxdc49b3be43c4733f83a2fe80ccbe6fee11c8ba12', 'bnUSD', '0x0'], 'data': []} 

{'scoreAddress': 'cxedda469d71f9d3cb76b5585e129a374f988684bc', 'indexed': ['AssetAdded(Address,str,bool)', 'cx23abbd450911b5f7feedda2de23621ac765622fb', 'BALN', '0x1'], 'data': []} 

{"governance|setAddresses|{'_addresses': {'loans': 'cxedda469d71f9d3cb76b5585e129a374f988684bc', 'staking': 'cx272ce34980099a03c2465c8de6f445ff86416793', 'dividends': 'cx37854d456b9095ccd82b39f5e3f0e60e6e7577f1', 'reserve': 'cxffa4b54daff78fe6cbb367dbea6fbd077e86c77b', 'daofund': 'cx7f5b9527a5186ae60262bad895d67

In [27]:
# Cell 7b
# Launch Balanced
# We may want to make this a payable method and have the governance SCORE borrow bnUSD,
# start and name the sICXbnUSD market, and add it as a rewards DataSource.

launch_results = launch_balanced(btest_wallet, staking_wallet)
print(launch_results)

------------------------------------------------------------------------------------------------------------------
Calling launchBalanced, with parameters {} on the governance contract.
------------------------------------------------------------------------------------------------------------------
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxedda469d71f9d3cb76b5585e129a374f988684bc', 'indexed': ['ContractActive(str,str)', 'Loans'], 'data': ['Active']} 

------------------------------------------------------------------------------------------------------------------
Calling toggleStakingOn, with parameters {} on the staking contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
------------------------------------------------------------------------------------------------------------------
Calling delegate, with parameters {'_delegations': [{'_address': 'hx9eec61296a7010c867ce24c20e69588e2832bc52', '_votes_in_per': '50000000000000000000'}, {'_address': 'hx000e0415037ae871184b2c7154e5924ef2bc075e', '_votes_in_per': '50000000000000000000'}]} on the governance contract.
------------------------------------------------------------------------------------------------------------------
************************************************** Status: 1 **************************************************
{'governance|launchBalanced|{}': {'txHash': '0x848f4c114fda6aebaad47657c41874edc1d00f402212be76c555f9d1473563b6', 'blockHeight': 770726, 'blockHash': '0x1101a94b5be446b60fa8375b1fa13386b25e538132b28392d8d7cb6f034a4211', 'txIndex': 1, 'to'

In [ ]:
# Cell 8
# Deploy or Update a single SCORE

contract_name = 'rewards'
update = 1
params = {}
if update == 0:
    params = {'_governance': contracts['governance']['SCORE']}

compress()
contract = contracts[contract_name]
confirm = input(f'{"Updating" if update else "Deploying"} {contract_name} with params: {params} to {network}. Proceed (Yes/No)? ')
if confirm == 'Yes':
    deploy_SCORE(contract, params, btest_wallet, update)

In [16]:
send_tx('loans', 500*ICX, 'depositAndBorrow', {'_asset': 'bnUSD', '_amount': 50* ICX}, btest_wallet)


------------------------------------------------------------------------------------------------------------------
Calling depositAndBorrow, with parameters {'_asset': 'bnUSD', '_amount': 50000000000000000000} on the loans contract.
------------------------------------------------------------------------------------------------------------------
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxb23ed1c0e174bed66a625507303f4a934eae1aca', 'indexed': ['ICXTransfer(Address,Address,int)', 'cxb23ed1c0e174bed66a625507303f4a934eae1aca', 'cxad32e2e33b3e7e97f2e6725e8dbd4424067e3dba', '0x1b1ae4d6e2ef500000'], 'data': []} 

{'scoreAddress': 'cxadfef188e7134700dd27304d016a85eb83767cd7', 'indexed': ['Mint(Address,int,bytes)', 'cxb23ed1c0e174bed66a625507303f4a934eae1aca'], 'data': ['0x1b1ae4d6e2ef500000', '0x4e6f6e65']} 

{'scoreAddress': 'cxadfef188e7134700dd27304d016a85eb83767cd7', 'indexed': ['Transfer(Address,Addres

{'txHash': '0xb9835077b0b859f90b47a5b49bd4856335c79ba5f276317e242daf707c64cc31',
 'blockHeight': 770117,
 'blockHash': '0x391b505b84fc04fe8273e2aa12de739bc80f2747d9d028af2071c2734c3a082d',
 'txIndex': 1,
 'to': 'cxb23ed1c0e174bed66a625507303f4a934eae1aca',
 'stepUsed': 876640,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 876640,
 'eventLogs': [{'scoreAddress': 'cxb23ed1c0e174bed66a625507303f4a934eae1aca',
   'indexed': ['ICXTransfer(Address,Address,int)',
    'cxb23ed1c0e174bed66a625507303f4a934eae1aca',
    'cxad32e2e33b3e7e97f2e6725e8dbd4424067e3dba',
    '0x1b1ae4d6e2ef500000'],
   'data': []},
  {'scoreAddress': 'cxadfef188e7134700dd27304d016a85eb83767cd7',
   'indexed': ['Mint(Address,int,bytes)',
    'cxb23ed1c0e174bed66a625507303f4a934eae1aca'],
   'data': ['0x1b1ae4d6e2ef500000', '0x4e6f6e65']},
  {'scoreAddress': 'cxadfef188e7134700dd27304d016a85eb83767cd7',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'hx0000000000000000000000000000000000000000',
    'cxb23e

Testing getBAlances method

In [28]:
send_tx('loans', 500*ICX, 'depositAndBorrow', {'_asset': 'bnUSD', '_amount': 50* ICX}, btest_wallet)
time.sleep(70)
for i in range(10):
    fast_send_tx('rewards', 0, 'distribute', {}, btest_wallet)
    
total_balances = 0
for contract in ['rewards', 'reserve', 'bwt', 'dex', 'daofund']:
    result = int(call_tx('baln', 'balanceOf', {'_owner': contracts[contract]['SCORE']}, False), 0)
    total_balances += result

print(f'Total BALN: {total_balances / 10**18}')
day = (total_balances / 10**18)/100000
res = call_tx('reserve', 'getBalances', {}, False)
print(res)
assert int(res['BALN'], 0)/10**18 == 5000* day, "Reserve not receiving proper rewards"
print('Test case success')

------------------------------------------------------------------------------------------------------------------
Calling depositAndBorrow, with parameters {'_asset': 'bnUSD', '_amount': 50000000000000000000} on the loans contract.
------------------------------------------------------------------------------------------------------------------
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxedda469d71f9d3cb76b5585e129a374f988684bc', 'indexed': ['ICXTransfer(Address,Address,int)', 'cxedda469d71f9d3cb76b5585e129a374f988684bc', 'cx272ce34980099a03c2465c8de6f445ff86416793', '0x1b1ae4d6e2ef500000'], 'data': []} 

{'scoreAddress': 'cx2f4b75d1eba58dc41dd45b3b1d09d33319f751c5', 'indexed': ['Mint(Address,int,bytes)', 'cxedda469d71f9d3cb76b5585e129a374f988684bc'], 'data': ['0x1b1ae4d6e2ef500000', '0x4e6f6e65']} 

{'scoreAddress': 'cx2f4b75d1eba58dc41dd45b3b1d09d33319f751c5', 'indexed': ['Transfer(Address,Addres

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
{'scoreAddress': 'cx23abbd450911b5f7feedda2de23621ac765622fb', 'indexed': ['Mint(Address,int,bytes)', 'cxc1b3632a24bdb243620bbffbf1b8dc8175346a27'], 'data': ['0x152d02c7e14af6800000', '0x4e6f6e65']} 

{'scoreAddress': 'cx23abbd450911b5f7feedda2de23621ac765622fb', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx0000000000000000000000000000000000000000', 'cxc1b3632a24bdb243620bbffbf1b8dc8175346a27', '0x152d02c7e14af6800000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx23abbd450911b5f7feedda2de23621ac765622fb', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxc1b3632a24bdb243620bbffbf1b8dc8175346a27', 'cx03198e44c6dff023caa943a543fad9a9ac63aa3a', '0x43c33c1937564800000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx23abbd450911b5f7feedda2de23621ac765622fb', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxc1b3632a24bdb243620bbffbf1b8dc8175346a27', 'cxffa4b54daff78fe6cbb367dbea6fbd077e86c77b', '0x10f0cf064dd59200000'], 'data': ['0x4e6f6e65']} 


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
{'scoreAddress': 'cxc1b3632a24bdb243620bbffbf1b8dc8175346a27', 'indexed': ['Report(int,str,int,int)', '0x1', 'Loans'], 'data': ['0x0', '0x0']} 



...

Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Total BALN: 100000.0
{'BALN': '0x10f0cf064dd59200000'}
Test case success


Testing redeem

In [29]:
res = send_tx('reserve', 0, 'redeem', {'_to': user1.get_address(), '_amount': 10* ICX, '_sicx_rate': 2}, btest_wallet)
assert res['failure']['message'] == 'BalancedReserveFund: The redeem method can only be called by the Loans SCORE.', 'Redeem methos can only be called from loans'
print('Test case passed')

------------------------------------------------------------------------------------------------------------------
Calling redeem, with parameters {'_to': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', '_amount': 10000000000000000000, '_sicx_rate': 2} on the reserve contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 0 **************************************************
Failure: {'code': 32, 'message': 'BalancedReserveFund: The redeem method can only be called by the Loans SCORE.'}
Test case passed


worker token

In [95]:
call_tx('baln', 'balanceOf', {'_owner': btest_wallet.get_address()}, False)

'0x2fa54641bae8aaa00000'

In [96]:
send_tx('governance', 0, 'balwAdminTransfer', {'_from': btest_wallet.get_address(), '_to': user1.get_address(), '_value': 10* ICX}, btest_wallet)


------------------------------------------------------------------------------------------------------------------
Calling balwAdminTransfer, with parameters {'_from': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', '_to': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', '_value': 10000000000000000000} on the governance contract.
------------------------------------------------------------------------------------------------------------------
************************************************** Status: 0 **************************************************
Failure: {'code': 1, 'message': 'Insufficient balance.'}


{'txHash': '0x95271abef101817c2ce3d7ac02bf96f855dc9b08827fb9c1dd3beed2d18269c5',
 'blockHeight': 737073,
 'blockHash': '0x334e305de16c0403d620eb55b4a5446b48e5ecf6326ee6cf806f0ad683765875',
 'txIndex': 1,
 'to': 'cxc3adcc3383cedfe483fb23a264a90955e604c2de',
 'stepUsed': 185000,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 185000,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [97]:
send_tx('bwt', 0, 'distribute', {}, btest_wallet)


------------------------------------------------------------------------------------------------------------------
Calling distribute, with parameters {} on the bwt contract.
------------------------------------------------------------------------------------------------------------------
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx8493aadc07137f6dd3832729799bf87535c89b36', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxd3cfdc52b4b62d0dd1abe1635a67dc63a75d64b2', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', '0x0'], 'data': ['0x4e6f6e65']} 



{'txHash': '0x2dc527d64d4d69822d168be4cdf27a6f6337c17d0b80d0ad1b0d8432f364b10f',
 'blockHeight': 737076,
 'blockHash': '0x02e04658c6769494435c683213ef34168619587e581a0bd0a52493e6018bef6f',
 'txIndex': 1,
 'to': 'cxd3cfdc52b4b62d0dd1abe1635a67dc63a75d64b2',
 'stepUsed': 190600,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 190600,
 'eventLogs': [{'scoreAddress': 'cx8493aadc07137f6dd3832729799bf87535c89b36',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'cxd3cfdc52b4b62d0dd1abe1635a67dc63a75d64b2',
    'hx3f01840a599da07b0f620eeae7aa9c574169a4be',
    '0x0'],
   'data': ['0x4e6f6e65']}],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\